<h1 align="center">PART II</h1>
<h1 align="center">Sentiment Analysis Classifications - Review and Comparison</h1>

First, we needed to create vector words. For simplicity, we used a pre-trained model.

Google was able to teach the Word2Vec model on a massive Google News dataset that contained over 100 billion different words! Google has created [3 million vector words](https://code.google.com/archive/p/word2vec/#Pre-trained_word_and_phrase_vectors) from this model, each with a dimension of 300.

Ideally, we would use these vectors, but because the vector-word matrix is quite large (3.6 GB), we used a much more manageable matrix, which was trained using [GloVe](https://nlp.stanford.edu/projects/glove/), with a similar model of vector word generation. This matrix contains 400,000 vector words, each with a dimension of 50. You can also download model [here](https://www.kaggle.com/anindya2906/glove6b?select=glove.6B.50d.txt).

#### How word2vec works:

The idea behind word2vec is that:

    Take a 3 layer neural network. (1 input layer + 1 hidden layer + 1 output layer)
    Feed it a word and train it to predict its neighbouring word.
    Remove the last (output layer) and keep the input and hidden layer.
    Now, input a word from within the vocabulary. The output given at the hidden layer is the ‘word embedding’ of the input word.
    
Two popular examples of methods of learning word embeddings from text include:

    Word2Vec
    GloVe

To get started, let's download the necessary libraries:

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim, logging
import gensim.models.keyedvectors as word2vec
import matplotlib.pyplot as plt

%matplotlib inline

Also let's write a style for alignment in the middle of all graphs, images, etc:

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

Next, we will load the sample data we processed in the previous part:

In [3]:
with open('documents.pql', 'rb') as f:
     docs = pickle.load(f)

In [4]:
print("Number of documents:", len(docs))

Number of documents: 38544


Now we will load our glove model in word2vec format. Because the GloVe dump from Stanford's site is slightly different from the word2vec format. You can convert a GloVe file to word2vec format using the following command in your console:

`python -m gensim.scripts.glove2word2vec --input  model/glove.6B.50d.txt --output model/glove.6B.50d.w2vformat.txt`

After that you can delete original GloVe model.

Next operation may take some time, as the model contains 400 000 words, so we will get a 400 000 x 50 embedding matrix that contains all the values of the word vectors.

In [5]:
model = word2vec.KeyedVectors.load_word2vec_format('model/glove.6B.50d.w2vformat.txt', binary=False)

Now let's get a list of all the words from our dictionary:

In [6]:
words = list(model.vocab)

Just to make sure everything is loaded correctly, we can look at the dimensions of the dictionary list and the embedding matrix:

In [7]:
print(words[:50], "\n\nTotal words:", len(words), "\n\nWord-Vectors shape:", model.vectors.shape)

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s", 'for', '-', 'that', 'on', 'is', 'was', 'said', 'with', 'he', 'as', 'it', 'by', 'at', '(', ')', 'from', 'his', "''", '``', 'an', 'be', 'has', 'are', 'have', 'but', 'were', 'not', 'this', 'who', 'they', 'had', 'i', 'which', 'will', 'their', ':', 'or', 'its', 'one', 'after'] 

Total words: 400000 

Word-Vectors shape: (400000, 50)


We can also find a word like "football" in our word list and then access the corresponding vector through the embedding matrix:

In [8]:
print(model['football'])

[-1.8209    0.70094  -1.1403    0.34363  -0.42266  -0.92479  -1.3942
  0.28512  -0.78416  -0.52579   0.89627   0.35899  -0.80087  -0.34636
  1.0854   -0.087046  0.63411   1.1429   -1.6264    0.41326  -1.1283
 -0.16645   0.17424   0.99585  -0.81838  -1.7724    0.078281  0.13382
 -0.59779  -0.45068   2.5474    1.0693   -0.27017  -0.75646   0.24757
  1.0261    0.11329   0.17668  -0.23257  -1.1561   -0.10665  -0.25377
 -0.65102   0.32393  -0.58262   0.88137  -0.13465   0.96903  -0.076259
 -0.59909 ]


<h2 align="center">Word Average Embedding Model</h2>

Well, let's start analyzing our vectors. Our first approach will be the **word average embedding model**. 

The essence of this naive approach is to take the average of all word vectors from a sentence to get one 50-dimensional vector that represents the tone of the whole sentence that we feed the model and try to get some quick result.

We didn't have to put a try/except, but even though I cleaned up our sample, there were a couple of words left after the processing that needed to be searched for and removed.

In [9]:
def sent_embed(words, docs):
    x_sent_embed, y_sent_embed = [], []
    count_words, count_non_words = 0, 0  
    
    # recover the embedding of each sentence with the average of the vector that composes it
    # sent - sentence, state - state of the sentence (pos/neg)
    for sent, state in docs:
        # average embedding of all words in a sentence
        sent_embed = []
        for word in sent:
            try:
                # if word is present in the dictionary - add its vector representation
                count_words += 1
                sent_embed.append(model[word])
            except KeyError:
                # if word is not in the dictionary - add a zero vector
                count_non_words += 1
                sent_embed.append([0] * 50)
        
        # add a sentence vector to the list
        x_sent_embed.append(np.mean(sent_embed, axis=0).tolist())
        
        # add a label to y_sent_embed
        if state == 'pos': y_sent_embed.append(1)
        elif state == 'neg': y_sent_embed.append(0)
            
    print(count_non_words, "out of", count_words, "words were not found in the vocabulary.")
    
    return x_sent_embed, y_sent_embed

In [10]:
x, y = sent_embed(words, docs)

30709 out of 1802696 words were not found in the vocabulary.


<h2 align="center">Cosine Similarity</h2>

To measure the similarity of 2 words, we need a way to measure the degree of similarity between 2 embedding vectors for these 2 words. Given 2 vectors $u$ and $v$, cosine similarity is determined as follows:

$$\text{cosine_similarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta)$$

where: 

* $u.v$ - dot product (or inner product) of two vectors;

* $||u||_2$ - norm (or length) of the vector $u$;
    
    * **Note**: norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$)

* $\theta$ is the angle between $u$ and $v$. 

This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

**`cosine_similarity()`** is a method that used to estimate the similarity between word vectors.

In [11]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # compute the dot product between u and v
    dot = np.dot(u,v)
    
    # compute the L2 norm of u
    norm_u = np.sqrt(sum(u**2))
    
    # Compute the L2 norm of v
    norm_v = np.sqrt(sum(v**2))
    
    # Compute the cosine similarity defined by formula above
    cosine_similarity = dot/(norm_u*norm_v)
    
    return cosine_similarity

Let's check the cosine similarity on 2 negative sentences:

In [12]:
print("Sentence #5: ", docs[5], "\n\nSentence #7: ", docs[7])
print("\nSentence Embedding #5: ", x[5], "\n\nSentence Embedding #7: ", x[7])

Sentence #5:  (['eager', 'read', 'book', 'reading', 'left', 'completely', 'flat', 'rest', 'iceberg', 'hardly', 'smith', 'barely', 'give', 'could', 'reading', 'sport', 'illustrated', 'minnesota', 'sport', 'page', 'admits', 'football', 'majority', 'book', 'leisurely', 'describes', 'basic', 'part', 'football', 'career', 'book', 'touted', 'full', 'complete', 'story', 'learned', 'watching', 'stimpy', 'draft', 'type', 'meaty', 'story', 'want', 'read', 'telling', 'inside', 'detail', 'smith', 'deal', 'people', 'wanted', 'friend', 'girlfriend', 'star', 'football', 'player', 'friend', 'could', 'trust', 'need', 'trust', 'thought', 'enter', 'mind', 'look', 'hire', 'agent', 'goofing', 'missing', 'class', 'summer', 'covered', 'story', 'time', 'ohio', 'state', 'begin', 'using', 'football', 'smart', 'enough', 'study', 'break', 'coaching', 'process', 'mentally', 'challenging', 'happened', 'plan', 'medical', 'school', 'know', 'story', 'football', 'part', 'iceberg', 'already', 'documented', 'also', 'felt

In [13]:
print("cosine_similarity = ", cosine_similarity(np.array(x[5]), np.array(x[7])))

cosine_similarity =  0.8968743967161681


A value of 0.89 indicates that the sentences are close to each other, and so it is.

Let's check on two positive sentences:

In [14]:
print("Sentence #1: ", docs[1], "\n\nSentence #4: ", docs[4])
print("\nSentence Embedding #1: ", x[1], "\n\nSentence Embedding #4: ", x[4])

Sentence #1:  (['diehard', 'graco', 'daughter', 'born', 'graco', 'snugride', 'rated', 'infant', 'seat', 'stock', 'consumer', 'report', 'infant', 'seat', 'great', 'part', 'travel', 'system', 'loved', 'stroller', 'seat', 'outgrew', 'seat', 'around', 'year', 'tried', 'eddie', 'bauer', 'convertible', 'seat', 'next', 'horrible', 'strap', 'always', 'twisted', 'daughter', 'head', 'always', 'fell', 'forward', 'onto', 'chest', 'fell', 'asleep', 'quickly', 'changed', 'graco', 'comfortsport', 'overhead', 'drop', 'loved', 'padding', 'head', 'rested', 'side', 'carseat', 'instead', 'chest', 'strap', 'never', 'twisted', 'easy', 'heavy', 'around', 'airport', 'outgrew', 'seat', 'decided', 'make', 'switch', 'britax', 'although', 'roundabout', 'rate', 'went', 'marathon', 'thing', 'like', 'infant', 'latch', 'need', 'child', '3yrs', 'higher', 'weight', 'limit', 'roundabout', 'honda', 'accord', 'door', 'husband', 'drive', 'door', 'toyota', 'echo', 'tiny', 'seat', 'perfectly', 'well', 'vehicle', 'forward', '

In [15]:
print("cosine_similarity = ", cosine_similarity(np.array(x[1]), np.array(x[4])))

cosine_similarity =  0.9481159093219256


These sentences are also close to each other. 

So now let's check sentences with different states:

In [16]:
print("Sentence #1: ", docs[0], "\n\nSentence #5: ", docs[6])

Sentence #1:  (['love', 'crate', 'smart', 'little', 'learned', 'zipper', 'undone', 'escape', 'course', 'home', 'find', 'small', 'gift', 'left', 'behind'], 'pos') 

Sentence #5:  (['product', 'generate', 'enough', 'heat', 'provide', 'relief', 'sore', 'aching', 'muscle', 'massaging', 'action', 'actually', 'vibration', 'action', 'heat', 'switched', 'vibration', 'action', 'reduced', 'dramatically'], 'neg')


In [17]:
print("cosine_similarity = ", cosine_similarity(np.array(x[0]), np.array(x[6])))

cosine_similarity =  0.7410293614966914


As we see, our average embedding still has some problems with separating different classes with cosine similarity.